In [22]:
from keras.models import Sequential
from keras import layers
from keras import optimizers
from keras import callbacks
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import tensorflow as tf
from tensorflow import keras

In [2]:
df = pd.read_csv("database_mscc.csv")

In [3]:
df.drop(columns = "Unnamed: 0", inplace=True)

In [4]:
for row in np.arange(0,df.shape[0]):
    df["y"][row] = df["y"][row][7:]

/var/folders/7d/cjq_vxdx7vqfc6jkyf5tpx4r0000gn/T/ipykernel_2570/4108488883.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["y"][row] = df["y"][row][7:]


In [5]:
df.isna().sum()

y         0
0         0
1         0
2         0
3         0
       ... 
224    1439
225    1439
226    1439
227    1439
228    1440
Length: 230, dtype: int64

In [6]:
df = df.fillna(0)

In [7]:
df["y"].drop_duplicates()

0     surprised
1       fearful
2       disgust
14        happy
15         calm
18      neutral
34          sad
35        angry
Name: y, dtype: object

In [8]:
y = df["y"]
X = df.drop(columns="y")

In [9]:
# Split to train, validation, and test sets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1405, stratify=y)

#'One-hot' vectors for Y: emotion classification
lb = LabelEncoder()
y_train_cat = np_utils.to_categorical(lb.fit_transform(y_train))
y_test_cat = np_utils.to_categorical(lb.fit_transform(y_test))

In [10]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
X_train_scaled_array = np.array(X_train_scaled)
X_test_scaled_array = np.array(X_test_scaled)
y_train_array = np.array(y_train_cat)
y_test_array = np.array(y_test_cat)

In [12]:
X_train_expanded_dims = np.expand_dims(X_train_scaled_array, axis=2)
X_test_expanded_dims = np.expand_dims(X_test_scaled_array, axis=2)

In [23]:
model = Sequential()
model.add(layers.LSTM(64, return_sequences = True, input_shape=(X_train.shape[1],1)))
model.add(layers.LSTM(64))
model.add(layers.Dense(8, activation = 'softmax'))
print(model.summary())

batch_size = 32

# Callbacks functions
checkpoint_path = 'database_mscc.csv'

#-> Save the best weights
"""mcp_save = callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True,
                           monitor='val_categorical_accuracy',
                           mode='max')"""
#-> Reduce learning rate after 100 epoches without improvement.
#rlrop = callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                    #factor=0.1, patience=100)

                           
# Compile & train   
"""model.compile(loss='categorical_crossentropy', 
                optimizer='RMSProp', 
                metrics=['accuracy'])

history = model.fit((X_train_expanded_dims, y_train_array, validation_split=0.3, epochs=500, batch_size=32,callbacks = [rlrop])"""
opt = keras.optimizers.Adam(lr=0.0001)

model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
es = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train_expanded_dims, y_train_array, validation_split=0.3, epochs=500, batch_size=32, callbacks=[es])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 229, 64)           16896     
                                                                 
 lstm_13 (LSTM)              (None, 64)                33024     
                                                                 
 dense_6 (Dense)             (None, 8)                 520       
                                                                 
Total params: 50,440
Trainable params: 50,440
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500


/Users/guilhermecavalcantidesabarreto/.pyenv/versions/3.8.12/envs/silvertone/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


23/23 [==============================] - 13s 383ms/step - loss: 2.0801 - accuracy: 0.1404 - val_loss: 2.0758 - val_accuracy: 0.1155
Epoch 2/500
23/23 [==============================] - 7s 319ms/step - loss: 2.0730 - accuracy: 0.1404 - val_loss: 2.0721 - val_accuracy: 0.1155
Epoch 3/500
23/23 [==============================] - 7s 320ms/step - loss: 2.0718 - accuracy: 0.1418 - val_loss: 2.0725 - val_accuracy: 0.1155
Epoch 4/500
23/23 [==============================] - 8s 333ms/step - loss: 2.0700 - accuracy: 0.1418 - val_loss: 2.0703 - val_accuracy: 0.1155
Epoch 5/500
23/23 [==============================] - 7s 325ms/step - loss: 2.0680 - accuracy: 0.1418 - val_loss: 2.0703 - val_accuracy: 0.1155
Epoch 6/500
23/23 [==============================] - 7s 322ms/step - loss: 2.0671 - accuracy: 0.1418 - val_loss: 2.0716 - val_accuracy: 0.1155
Epoch 7/500
23/23 [==============================] - 7s 316ms/step - loss: 2.0656 - accuracy: 0.1418 - val_loss: 2.0725 - val_accuracy: 0.1155
Epoch 8/50

KeyboardInterrupt: 